In [ ]:
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')
        
def run_elastix(fixed_image, moving_image, params):
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

##  Translate the ct image by a large amount and see if we can recover the offset using registration.

In [ ]:
fixedImage = sitk.ReadImage('sampleImages/ct_2d_moving.tif')
movingImage= sitk.ReadImage('sampleImages/ct_translated.tif')

Mouse over the images below, looking at the "Position" displayed at the bottom. Pick a point that you can find in both images, and make note of its position. 

* What do you think the correct translation is?

In [ ]:
itkwidgets.compare(movingImage,fixedImage)

Let's first try running registration with default parameters for 'translation'.

In [ ]:
params = sitk.ReadParameterFile('elastixParameters/AffineMI_2d.txt')
sitk.PrintParameterMap( params )

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params)
transformedMovingImage = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(transformedMovingImage,fixedImage)

In [ ]:
params = sitk.GetDefaultParameterMap('translation')
params['MaximumNumberOfIterations'] = ['2000']
sitk.PrintParameterMap(params)

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params)
transformedMovingImage = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(transformedMovingImage,fixedImage)

In [ ]:
params['MaximumNumberOfIterations'] = ['128']
params['AutomaticTransformInitialization'] = ['true']
params['AutomaticTransformInitializationMethod'] = ['CenterOfGravity']
sitk.PrintParameterMap(params)

In [ ]:
elastixImageFilter = run_elastix(fixedImage, movingImage, params)
transformedMovingImage = elastixImageFilter.GetResultImage()

In [ ]:
itkwidgets.compare(transformedMovingImage,fixedImage)

The lines below will print the translation of the moving image to match the fixed image.

* Predict what the translation will be

In [ ]:
transformation = elastixImageFilter.GetTransformParameterMap()
transformation[0]['TransformParameters']

* Does the output match your prediction? 
* If no, why?